# Cricket Match Votes

In [1]:
import numpy as np
import pandas as pd
import json

# Api given script for collect json format data using python
import http.client

conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}

conn.request("GET", "/api/cricket/match/13265827/votes", headers=headers)

res = conn.getresponse()
data = res.read()


# end api script

data_json = json.loads(data.decode("utf-8"))

# Save to a file
with open("api_data.json", "w", encoding="utf-8") as f:
    json.dump(data_json, f, indent=4)

print("Data saved as api_data.json")

Data saved as api_data.json


## Loading Json file

In [2]:
with open("api_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)
data

{'vote': {'vote1': 2336, 'vote2': 1636, 'voteX': None},
 'bothTeamsToScoreVote': {'voteYes': 2, 'voteNo': 0},
 'firstTeamToScoreVote': {'voteHome': 2, 'voteNoGoal': 0, 'voteAway': 0},
 'whoShouldHaveWonVote': {'vote1': 0, 'vote2': 0}}

# Depth json to flatten_json

In [3]:
# Function for Convert a nested json dictionary to flatten json or dictionary
def flatten_json(nested_json, parent_key='', sep='_'):

    items = []
    
    # If the value is a dictionary
    if isinstance(nested_json, dict):
        for key, value in nested_json.items():
            new_key = f"{parent_key}{sep}{key}" if parent_key else key
            items.extend(flatten_json(value, new_key, sep=sep).items())
    
    # If the value is a list
    elif isinstance(nested_json, list): 
        for index, item in enumerate(nested_json):
            new_key = f"{parent_key}{sep}{index}" if parent_key else str(index)
            items.extend(flatten_json(item, new_key, sep=sep).items())
            
    # If the value is neither dict nor list
    else:
        items.append((parent_key, nested_json))

    return dict(items)

# Flatten Json to Row column

In [4]:
def process_json_file(data):
    try:
        # Normalize JSON data
        if isinstance(data, list):  # JSON is an array of objects
            flat_data = [flatten_json(item) for item in data]
        elif isinstance(data, dict):  # JSON is a single dictionary
            flat_data = [flatten_json(data)]
        else:
            raise ValueError("Unsupported JSON format")

        # Convert to DataFrame
        df = pd.DataFrame(flat_data)
        return df


    except json.JSONDecodeError:
        print("Error: Invalid JSON file.")
    except Exception as e:
        print(f"Error: {str(e)}")

## 1. first level keys

In [5]:
# first level keys
print("Data Type: ", type(data))
data.keys()

Data Type:  <class 'dict'>


dict_keys(['vote', 'bothTeamsToScoreVote', 'firstTeamToScoreVote', 'whoShouldHaveWonVote'])

In [6]:
data

{'vote': {'vote1': 2336, 'vote2': 1636, 'voteX': None},
 'bothTeamsToScoreVote': {'voteYes': 2, 'voteNo': 0},
 'firstTeamToScoreVote': {'voteHome': 2, 'voteNoGoal': 0, 'voteAway': 0},
 'whoShouldHaveWonVote': {'vote1': 0, 'vote2': 0}}

# simple data, so easily we can create dataframe

In [7]:
data = process_json_file(flatten_json(data))
data

,vote_vote1,vote_vote2,vote_voteX,bothTeamsToScoreVote_voteYes,bothTeamsToScoreVote_voteNo,firstTeamToScoreVote_voteHome,firstTeamToScoreVote_voteNoGoal,firstTeamToScoreVote_voteAway,whoShouldHaveWonVote_vote1,whoShouldHaveWonVote_vote2
0,2336,1636,None,2,0,2,0,0,0,0


In [8]:
# Saving file
data.to_excel('cricket_match_Votess.xlsx', index=False)